# Advanced Pandas concepts

This notebook introduces advanced Pandas concepts while revisiting basics such as indexing, filtering, and utilizing Seaborn for graph creation. We'll then explore reshaping data using `melt` and `pivot` to convert between wide and long data formats, delve into the `explode` function for nested data structures, and cap it off with a comprehensive exercise combining some of these concepts.

First, we'll import the necessary libraries.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import answers

## Mammal Body Size
(adapted from [Data Carpentry for Biologists](https://datacarpentry.org/semester-biology/))

There were a relatively large number of extinctions of mammalian species roughly 10,000 years ago. To help understand why these extinctions happened scientists are interested in understanding whether there were differences in the body size of those species that went extinct and those that did not.

The dataset `'data/mammal.tsv'` is the largest dataset on mammalian body size in the world. This dataset has data on the mass of recently extinct mammals as well as extant mammals (i.e., those that are still alive today). Take a look at the [original dataset's metadata](https://web.archive.org/web/20191219110715/http://www.esapubs.org/archive/ecol/E084/094/metadata.htm) to understand the structure of the data. One key thing to remember is that species can occur on more than one continent, and if they do then they will occur more than once in this dataset. 

Let’s explore the data.

### Exercise 1

Load the data from `mammal.tsv` into a dataframe. Keep in mind the following:

1. The data in `mammal.tsv` is tab separated. The character for this is `'\t'`.
2. `mammals.tsv` does not include a header. Use the link to the dataset's metadata above to find proper names for each of the columns.
3. By default Pandas' `read_csv()` will recognise certain strings as NaN-values, among which is the string `"NA"`. Since one of the continents in our data is `"NA"` (`"North America"`), we would like to disable this behaviour. This can be done by using the optional `read_csv()` argument `keep_default_na=False`.
4. Missing data in this file is specified by `-999` and `-999.00`. Tell Pandas that these are null values using the following optional `read_csv()` argument: `na_values=['-999', '-999.00']`. This will stop them from being plotted.
5. The data contains entries marked as *historical*, those can be considered *extinct*. It also containse entries marked *introduction*, those can be considered *extant*. You can use the pandas method `replace()` to change those values in your `DataFrame`.


In [ ]:
# your code here
display(df)

The resulting Dataframe should have 5731 rows and 9 columns.

### Exercise 2

Determine the number of different animals in the dataset. An animal is uniquely defined by the combination of its genus name and its species name. Print the result to the screen. The number should be somewhere between 4000 and 5000.

**HINT:** First, create a new column in which the strings of the genus and species columns are combined. Then, count the number of unique entries in this column. 

In [ ]:
# your code here

### Exercise 3

Find out how many of the animals are extinct and how many are extant, print the result to the screen. 

**HINT:** First separate the data into the extinct and extant components and then count the number of species.

In [ ]:
# your code here

**Do the numbers of extinct and extant add up to your answer for the number of animals? Why is this the case?**

_Your answer here._

### Exercise 4

Now print the genus name, the species name, and the mass of the heaviest and lightest animal. Note that it is not possible for a mammal to have negative mass. 

In [ ]:
# your code here

### Exercise 5

Use Seaborn to create a plot that shows the average body size for extinct species versus extant species. Make sure that is directly clear from the plot which is which. What kind of plot should you use here?

**HINT:** You do not need to use any Pandas operations here, as Seaborn does the data aggregation for you!

In [ ]:
# your code here

## Data formats: "wide" vs "long"

In the context of data organization, a table in "wide format" refers to a tabular structure where each observation or entity is represented by a _single_ row, and each variable or feature has _its own_ dedicated column. The following table is an example of a dataset in wide format:

| Student | English | History | Math |
| ------- | ------- | ------- | ---- |
| John    | 88      | 75      | 90   |
| Alice   | 80      | 78      | 85   |
| Bob     | 92      | 80      | 88   |

The wide format is intuitive, compact, and easy to understand. However, it might become less convenient for certain types of analyses or visualizations, especially when you need to compare or manipulate specific variable values across observations. For example, it would be a hassle to calculate the average grade over all subjects in the wide format table above, especially when there is missing data from students not following a specific course!

A different data format is the "long format". In long format, a variable and its values are represented in two columns. One column is used for the variable name and another is used for the corresponding value. The wide format table with student grades from above can also be displayed in long format:

| Student | Subject | Grade |
| ------- | ------- | ----- |
| John    | English | 88    |
| Alice   | English | 80    |
| Bob     | English | 92    |
| John    | History | 75    |
| Alice   | History | 78    |
| Bob     | History | 80    |
| John    | Math    | 90    |
| Alice   | Math    | 85    |
| Bob     | Math    | 88    |

Each row in the long format now represents a unique combination of "Student," "Subject," and "Grade". Student is called a identifying column, the column Subject holds the names of the observed variables, and the column Grade holds the corresponding values. Observe that the long format requires more storage space to store the same data, as it stores the student's names multiple times.

> Certain operations (especially those involving grouping and reshaping) might be computationally more expensive in long format compared to wide format. Other operations could be computationally more expensive when the data is configured in wide format. It's essential to choose the data format that best suits your needs. In many cases, there is a trade-off between wide and long formats, and the choice between the two depends on the specific analyses, visualizations, or models you plan to apply to the data. It's often good practice to keep both formats in mind and choose the one that makes a particular task more straightforward.

Pandas (of course) has functions to convert from wide to long format, and the other way around. These functions are `melt()` and `pivot_table()` respectively.

## Melt

Pandas' `melt` function can be used to convert parts of a DataFrame with wide format data into long format. This is particularly useful if there are multiple columns that have the same type of content, such as the grades of the students for each of the different subjects above. 

When melting a DataFrame, you can use the following parameters:
- `id_vars`: this is an optional list of column names that identify a row. These columns will not be touched by melt, but will be included in the resulting DataFrame. In our example, this is the column "Student".
- `value_vars`: this is an optional list of all column names that should be used to retrieve variable values from. If `value_vars` is not specified, melt will use all columns that are not set as `id_vars`. In our example, these are all columns except for "Student", so we do not need to specify it.
- `var_name`: the name to use for the variable column. In our example, this is "Subject".
- `value_name`: the name to use for the observed values of the variables. In our example, this is "Grade".

The code below will convert the example dataframe from wide format to long format using melt:

In [ ]:
# Create example DataFrame with student grades for different subjects
df_wide = pd.DataFrame({
    'Student': ['Alice', 'Bob', 'John'],
    'English': [88, 80, 92],
    'History': [75, 78, 80],
    'Math': [90, 85, 88]
})

print("DataFrame in wide format:")
display(df_wide)

# Convert DataFrame to long format using melt
df_long = pd.melt(df_wide, id_vars=['Student'], var_name='Subject', value_name='Grade')

print("\nDataFrame in long format:")
display(df_long)

We can now use this long format dataframe to plot something that would be more difficult to plot with a wide format dataframe.

In [ ]:
# Create a grouped bar chart
sns.barplot(x='Subject', y='Grade', hue='Student', data=df_long)

# Add labels and title
plt.xlabel('Subject')
plt.ylabel('Score')
plt.title('Student Scores by Subject')

# Show the plot
plt.show()

### Exercise 6: Melt

Consider the DataFrame named `sales` that represents monthly sales data for different products in a retail store. The DataFrame has columns for the month, some product categories and their corresponding sales amounts, the location of the sale, and the salesperson. 

Your task is to reshape this data using `melt`. The resulting DataFrame should include the "Month" as an identifier, the product categories melted into a new pair of columns labeled "Product" and "Sales". The additional columns, such as the "Location" of the sale and the "Salesperson," should not be included in the reshaped DataFrame.

In [ ]:
sales = pd.DataFrame({
    'Month': ['Jan', 'Feb', 'Mar'],
    'Electronics': [120, 150, 180],
    'Clothing': [80, 90, 100],
    'Books': [30, 40, 50],
    'Location': ['Downtown', 'Uptown', 'Downtown'],
    'Salesperson': ['Bert', 'Yolanda', 'Bert']
})

# your code here

display(melted_sales)

In [ ]:
answers.test_melt(melted_sales)

## Pivot

Pandas' `pivot` function can be used to convert parts of a DataFrame with long format data into wide format. It takes unique values from a specified column and spreads them out into new columns, using another column to fill in the values. It does the exact opposite of the `melt` function.

When pivoting a DataFrame, you can use the following parameters:
- `columns`: the name (or a list of names) of the column(s) in your DataFrame from which the wide format column names should be derived. In our grades table example, this is the column "Subject", which will be spread out into "Math", "English", and "History" again.
- `index`: the name (or a list of names) of the column(s) that should be used as the index in the new DataFrame. In our case, this should be "Student".
- `values`: the name (or a list of names) of the column(s) that will populate the value of the columns of the new DataFrame. In our case, this should be "Grade".

When using numerical data, instead of using `pivot`, we often use `pivot_table`. This function performs the same tasks as `pivot` and uses the same parameters, but is able to handle duplicate entries and missing values. By default, `pivot_table` will take the average of all duplicate entries and fill any values with `NaN`.

> Note that pivoting could have some unwanted side-effects, as per the code below.

In [ ]:
df_wide_pivot = pd.pivot_table(df_long, columns='Subject', index='Student', values='Grade')

print('DataFrame after pivot:')
display(df_wide_pivot)

# The column subject is now the name of the columns, which we do not want
df_wide_pivot.columns.name = None

# We've used Student as index, but we want this to be part of the data
df_wide_pivot = df_wide_pivot.reset_index()

print('Dataframe after resetting index and removing column names')
display(df_wide_pivot)

# We can test that the melted and then pivoted DataFrame is the same as the original DataFrame
pd.testing.assert_frame_equal(df_wide_pivot, df_wide, check_dtype=False)

### Exercise 7: Pivot

For this exercise, a long-format DataFrame has been supplied, containing grades for imaginary students in specific courses. As is common with long-format data, not every student has a recorded grade for every course

Use `pivot` to create a wide format DataFrame named `pivot_grades` that has the student names as rows and the course names as columns. The values in the table should be the grades of the corresponding combination of student and course. 

> When a students only has a grade for one of the two courses, `pivot` will automatically assign the value `NaN`. You can leave these in.

In [ ]:
grades = pd.DataFrame([["Pascal", "Programming 2", 7.0], ["Morty", "Programming 1", 5.5], 
                       ["Slartibartfast", "Programming 1", 6.5], ["Ursula", "Programming 1", 9.5],
                       ["Morty", "Programming 2", 3.5], ["Marge", "Programming 1", 8.0],
                       ["Ursula", "Programming 2", 9.0]], 
                       columns = ["student_name", "course_name", "grade"])

display(grades)

# your code here

display(pivot_grades)

In [ ]:
answers.test_pivot(pivot_grades)

### Exercise 8: Pivot table

Here, we present another typical example of long-format data. The DataFrame contains (fake) sensor data for two cities in the Netherlands. Each row in the DataFrame represents a measurement, including information such as date, time, location, temperature, and humidity. Frequently, we need to aggregate this kind of data in order to be able to do more specific analyses or generate meaningful graphs. In this particular case, our focus is on identifying the highest temperature recorded on a specific date.

> As mentioned earlier, the default aggregation performed by `pivot_table` is taking the average. You can modify the aggregation function used by `pivot_table` through the optional parameter `aggfunc`. 

Use `pivot_table` to create a wide format DataFrame that has the dates as rows and the cities as columns. The values in the table should be the highest temperature recorded on the corresponding combination of city and date. 

In [ ]:
temperatures = pd.DataFrame({
    'Date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'],
    'Time': ['01:00', '01:00', '13:00', '13:00', '01:00', '01:00'],
    'City': ['De Bilt', 'Breda', 'De Bilt', 'Breda', 'De Bilt', 'Breda'],
    'Temperature': [20, 22, 25, 30, 22, 25],
    'Humidity': [45, 65, 60, 65, 55, 70]
})
display(temperatures)

# your code here

display(pivot_temperatures)

In [ ]:
answers.test_pivot_table(pivot_temperatures)

## Explode

The `explode` function in pandas is a powerful tool for handling nested or list-like data structures within a DataFrame. When applied to a column containing a list, `explode` transforms each element into a separate row, duplicating the values in all other columns. 

In [ ]:
df = pd.DataFrame({
    'ID': [1, 2, 3],
    'Items': [['A', 'B'], ['C'], ['D', 'E', 'F']],
    'Value': [12, 48, -36]
})

display(df)

# Using explode to transform the DataFrame
df_exploded = df.explode('Items')

display(df_exploded)

### Exercise 9: Explode

Provided is the DataFrame `movies`, containing information about various movies and their cast. The `movies` DataFrame has two columns: "movie", which represents the movie's title, and "actors", which contains a comma-separated list of actors starring in each respective movie.

Create an unnested version of the `movies` DataFrame below. Store the result in `exploded_actors`. 
  
**HINT:** Write a function `split_actors(actor_str)` that uses the `split(',')` method to split a single string of comma separated actors into a list of actors. You can then use this function in Pandas' `.map()` to get a list of actors in every row of your `DataFrame`!

In [ ]:
movies = pd.DataFrame([["Hababam Sinifi", "Kemal Sunal,Münir Özkul,Halit Akçatepe,Tarik Akan"],
                      ["The Shawshank Redemption","Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler"],
                      ["Aynabaji","Chanchal Chowdhury,Masuma Rahman Nabila,Bijori Barkatullah,Partha Barua"],
                      ["The Godfather", "Marlon Brando,Al Pacino,James Caan,Diane Keaton"]],
                     columns = ["movie", "actors"])

# your code here
display(exploded_actors)

In [ ]:
answers.test_explode(exploded_actors)

### Exercise 10: Combining several methods

As a final challenge try to see if you can combine the pandas methods `read_csv`, `to_csv`, `map`, `explode`, `sort_values`, `groupby` and `head` for a more complex data transformation.

The file `data/recipes.csv` contains a few recipes with their lists of ingredients. The recipes were rated on a scale from 1 to 5 by the users of a food website. The file also contains the average rating of the recipes. The contents:

    recipe_name,rating,ingredients
    Caprese Salad,4.5,tomato;olive oil;basil;mozzerella
    Lasagna,4.8,beef;pork;bacon;onion;celery;carrot;wine;tomato
    Beef Bourguignon,4.3,beef;bacon;onion;carrot;celery;flour;garlic;wine
    Hamburger,3.8,beef;bacon;letuce;bread;onion;mayo;ketchup;pickle
    Lentil Burger,4.0,lemon;lentils;yogurt;garlic;mushrooms;miso;parika;flour;bread;pickles

As you can see, the first column is the name, the second column contains the average rating and the third column contains all the ingredients (separated by a semicolon `;`).

We want to know for each ingredient, what the **top two recipes** are that contain that ingredient. So, the two recipes with the highest rating containing that ingredient. For example, the top two recipes for *bacon* are *Lasagna* and *Beef Bourguignon*. (*Hamburger* also contains *bacon*, but it's rating is lower than the other two recipes, so we ignore that one.)

Write a piece of code that reads the file `data/recipes.csv`, does all the required transformations and produces a file `ingredients.csv` into the `data` folder, containing the information we want. A fragment of the contents of the output file you should produce is shown here:

    ingredients,recipe_name
    bacon,Lasagna
    bacon,Beef Bourguignon
    basil,Caprese Salad
    beef,Lasagna
    beef,Beef Bourguignon
    bread,Hamburger
    bread,Lentil Burger
    carrot,Lasagna
    carrot,Beef Bourguignon
    ...

As you can see, the first two entries are the top two recipes for *bacon*. After that we see only one entry for *basil*, because *basil* occurs in only one recipe. This is okay: If an ingredient only occurs in one recipe, the output should only contain that entry.

_The output should be sorted by the name of the ingredient._ Note that the output csv should not contain indices. Your outputted csv should look exactly like the example above.

In [ ]:
# your code here

In [ ]:
answers.test_final()